In [1]:
import re
import os

volumeinfo =  'Selections from Lucian. Smith, Emily James, translators. New York; Harper Brothers, 1892.'


refsbook = ['<refsDecl n="CTS">',
  '<cRefPattern matchPattern="(\w+).(\w+)"',
   'replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\']/tei:div[@n=\'$2\'])">',
   '<p>This pointer pattern extracts book and section</p>',
  '</cRefPattern>',
  '<cRefPattern matchPattern="(\w+)"',
   'replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\'])">',
   '<p>This pointer pattern extracts book</p>',
  '</cRefPattern>',
 '</refsDecl>']

refssection = ['<refsDecl n="CTS">',
   '<cRefPattern matchPattern="(\w+)"',
    'replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\'])">',
    '<p>This pointer pattern extracts section.</p>',
   '</cRefPattern>',
  '</refsDecl>']

hathioffset = 18


tgroup = 'tlg0062'
edition = 'perseus-eng5'
worktitles = {}
hasbooks = {}
firstpages = {}
metadata = {}

worklines = {}


def normcase(s):
    stopwords = ['a','an','and','in','is','of','on','or','that','the','with']
    returns  = ''
    
    curwork = ''
    for foo in s.split():
        foo = foo.lower()
        if(not foo in stopwords):
            foo = re.sub('^(.)',foo[0].upper(),foo)
        returns = returns + ' ' + foo
        
    returns = re.sub('^\s*(.+)\s*$','\g<1>',returns)        
    ucase = returns[0].upper() 
    returns = re.sub('^(.)',returns[0].upper(),returns)
    return(returns)

def add2teiheader(l,teiheader):
    
    if(re.search('<name xml:lang="en">H. W. Fowler</name>',l)):
        teiheader.append('<name>Henry Watson Fowler</name>')
        return
    
    if(re.search('<name xml:lang="en">F. G. Fowler</name>',l)):
        teiheader.append('<name>Francis George Fowler</name>')
        return
    
    if(re.search('<resp>',l)):
        teiheader.append('<resp>Digital conversion and editing</resp>')
        return
    
    l = re.sub('<editor>','<editor role="translator">',l)
    
    if(re.search('</teHeader',l)):
        teiheader.append('<revisionDesc>')
        teiheader.append('<change when="2023-08-01" who="Gregory Crane">GRC found this file largely tagged but cannot find a record (or remember) how it was originally entered. He did the final work to make this compatible with Scaife.</change>')
        teiheader.append('</revisionDesc>')
    
    teiheader.append(l)
    #if(re.search('Library</authority',l)):
    #    teiheader.append('<idno type="filename">IDNO</idno>')

def splitvol(volnum):
    global worklines
    global volumeinfo
    fname = 'emilysmith.xml'
    f = open(fname)
    curpage = ''
    curwork = ''
    sawtitle = 0
    intext = 0
    teiheader = []
    worklines = {}
    
    curpagelink = 'https://hdl.handle.net/2027/hvd.hw3j0n?urlappend=%3Bseq=PNUM'
    
    for l in f:
        l = re.sub('\s+$','',l)
        
        if(re.search('<text',l)):
            intext = 1
        
        if(not intext):
            l = re.sub('target="(https://hdl.handle[^"]+)','target="'+curpagelink,l)
            if(re.search('<title ',l)):
                if(not sawtitle):
                    l = '<title xml:lang="eng">WORKTITLE</title>'
                    sawtitle = 1
            add2teiheader(l,teiheader)
            continue
            
        m = re.search('<pb n="([0-9]+)',l)
        if(m):
            curpage = m[1]
            
        if(re.search('subtype="book"',l)):
            hasbooks[curwork] = 1
            
        m = re.search('(tlg006[12].tlg[0-9]+)',l)
        if(m):
            curwork = m[1]
            metadata[curwork] = volumeinfo
            if(curpage and not curwork in firstpages):
                firstpages[curwork] = curpage
            else:
                print('nofirstpage',curwork)
            #print(curwork,curpage)
            
        m =  re.search('<head>([^<]+)',l)
        if(m and not curwork in worktitles):
            curhead = normcase(m[1])
            curhead = re.sub('\.$','',curhead)
            worktitles[curwork] = curhead
            #print(volnum,curwork,curhead)
    
    f.close()
    
    
    transbase = '<div type="translation" n="urn:cts:greekLit:tlg0062.tlg001.perseus-eng5" xml:lang="eng">'
    trans = ''
    f = open(fname)
    
    curwork = ''
    
    for l in f:
        l = re.sub('\s+$','',l)
        m = re.search('(tlg006[12]\.tlg[0-9]+)',l)
        if(m):
            curwork = m[1]
            worklines[curwork] = []
            worklines[curwork].append('<text xml:lang="eng">')
            worklines[curwork].append('<body>')
            trans = re.sub('tlg006[12]\.tlg001',curwork,transbase)
            worklines[curwork].append(trans)
            continue
        if(re.search('</TEI>',l)):
            continue
            
        if(curwork):
            worklines[curwork].append(l)
            
            
    
    f.close()
    
    for curwork in worklines:
        xmlfile = curwork + '.' + edition + '.xml'
        outfname = '/Users/gcrane/github/first1kwork/data/' + re.sub('(tlg006[12])\.(tlg[0-9]+)','\g<1>/\g<2>/',curwork) + xmlfile
        print(outfname)
        outf = open(outfname,'w')

        curbook = ''
        cursection = ''
        inrefsdecl = 0
        for l in teiheader:
            l = re.sub('<date>2023','<date type="release">2023-08-01',l)
            l = re.sub('WORKTITLE',worktitles[curwork],l)
            curseq = str(hathioffset + int(firstpages[curwork]))
            l = re.sub('PNUM',curseq,l)
            l = re.sub('IDNO',xmlfile,l)
            if(re.search('<refsD',l)):
                inrefsdecl = 1
                if(curwork in hasbooks):
                    curf = open('bookref.txt')
                    for line in curf:
                        print(line,file=outf,end='')
                else:
                    curf = open('sectionref.txt')
                    for line in curf:
                        print(line,file=outf,end='')
                continue
            if(re.search('</refsD',l)):
                inrefsdecl = 0
                continue
            if(inrefsdecl):
                continue
                
            
            print(l,file=outf)
        curpage = ''
        for l in worklines[curwork]:
            
            m = re.search('<pb n="([0-9]+)"',l)
            if(m):
                curpage = 'p.' + m[1]
                l = re.sub('<pb n="([0-9]+)"','<pb n="'+curpage+'"',l)
                
            l = re.sub('(<note[^>*])n="([0-9]+)\.([0-9]+)"','\g<1>n="p.\g<2>.n.\g<3>"',l)
            
            while(re.search('(<note[^>]*>)(.+?</note>)',l)):
            
                m = re.search('(<note[^>]*>)(.+?</note>)',l)
                if(m):
                    notetext = m[2]
                    if(re.search('[α-ω]',notetext)):
                        l = re.sub('(<note)([^>]*>.+?</note>)','<xnote xml:lang="mul"\g<2>',l,1)
                    else:
                        l = re.sub('(<note)([^>]*>.+?</note>)','<xnote xml:lang="eng"\g<2>',l,1)
            l = re.sub('<xnote','<note',l)
            
            #l = re.sub('<note','<note xml:lang="eng"',l)
            m = re.search('<div[^>]+subtype="book" n="([^"]+)"',l)
            if(m):
                curbook = m[1]
                l = re.sub('(subtype="[^"]+")','\g<1> xml:base="urn:cts:greekLit:'+curwork+'.'+edition+'"',l)
                
            m = re.search('<div[^>]+subtype="section" n="([^"]+)"',l)
            if(m):
                cursection = m[1]
                l = re.sub('(subtype="[^"]+")','\g<1> xml:base="urn:cts:greekLit:'+curwork+'.'+edition+':'+curbook+'"',l)
                
            print(l,file=outf)
            
        print('</body></text></TEI>',file=outf)
        outf.close()

        


for i in range(1,2):
    splitvol(str(i))
    
    
for i in range(1,71):
    if(i<10):
        curwork = 'tlg0062.tlg00' + str(i)    
    elif(i<100):
        curwork = 'tlg0062.tlg0' + str(i)
        
    if(not curwork in worktitles):
        print('\n'+curwork,'skipped')
        continue
        
        #print(curwork,edition,worktitles[curwork],metadata[curwork])
    #else:
    newfname = 'ctsfiles/' + curwork + '-cts.xml'
    if(os.path.exists(newfname)):
        print('exists',newfname)
    else:
        xmlfile = curwork + '.' + edition + '.xml'

        fname = '/Users/gcrane/github/canonical-greekLit/data/tlg0062/' + re.sub('tlg006[12]\.(tlg[0-9]+)','\g<1>/',curwork) + '__cts__.xml'

        f = open(fname)
    
        text = f.read()
        f.close()
        outf = open(newfname,'w')
        outf.write(text)
        print('created',newfname)
    
    outfname = '/Users/gcrane/github/first1kwork/data/' + re.sub('(tlg006[12])\.(tlg[0-9]+)','\g<1>/\g<2>/',curwork) + '__cts__.xml'

    f = open(newfname)
    text = f.read()
    print('start',newfname,text)
    outf = open(outfname,'w')
    
            
    fullmeta = '<ti:translation urn="urn:cts:greekLit:'+curwork+'.perseus-eng5" xml:lang="eng" workUrn="urn:cts:greekLit:'+curwork+'">\n'
    fullmeta = fullmeta + '<ti:label xml:lang="eng">' + worktitles[curwork] + '</ti:label>\n'
    fullmeta = fullmeta + '<ti:description xml:lang="eng">' + metadata[curwork] + '</ti:description>\n\n</ti:translation>\n\n</ti:work>\n'
    newtext = re.sub('</ti:work>',fullmeta,text)
    outf.write(newtext)
    outf.close()
    print('wrote',outfname,newtext)
    #print('\n'+text+'\n')

/Users/gcrane/github/first1kwork/data/tlg0062/tlg012/tlg0062.tlg012.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0062/tlg029/tlg0062.tlg029.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0062/tlg018/tlg0062.tlg018.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0062/tlg024/tlg0062.tlg024.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0062/tlg019/tlg0062.tlg019.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0062/tlg016/tlg0062.tlg016.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0062/tlg044/tlg0062.tlg044.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0061/tlg001/tlg0061.tlg001.perseus-eng5.xml
/Users/gcrane/github/first1kwork/data/tlg0061/tlg004/tlg0061.tlg004.perseus-eng5.xml

tlg0062.tlg001 skipped

tlg0062.tlg002 skipped

tlg0062.tlg003 skipped

tlg0062.tlg004 skipped

tlg0062.tlg005 skipped

tlg0062.tlg006 skipped

tlg0062.tlg007 skipped

tlg0062.tlg008 skipped

tlg0062.tlg009 skipped

tlg0062.tlg010 ski

In [8]:
for foo in worktitles:
    print(foo,worktitles[foo],sep='\t')

	Introduction
tlg0062.tlg029	The Dream
tlg0062.tlg018	Zeus the Tragedian
tlg0062.tlg024	The Sale of Lives
tlg0062.tlg019	The Cock
tlg0062.tlg016	The Ferry
tlg0062.tlg012	A True History
tlg0062.tlg044	Toxaris; Or, Friendship
tlg0061.tlg001	Loukios; Or, the Ass
tlg0061.tlg004	The Halcyon
